In [1]:
class Screen(object):

    # Construtor da classe
    def __init__(self, title, bgColor, width, height):
        self.title = title       # título da janela
        self.bgColor = bgColor   # cor de fundo
        self.width = width       # largura da janela
        self.height = height     # altura da janela
        self.screen = pygame.display.set_mode(self.size()) # define o tamanho da tela
        pygame.display.set_caption(self.title)             # define o título da janela
        
    # Executa o pipeline gráfico
    def run(self, obj):
        while True:  # laço principal
            # captura eventos
            for event in pygame.event.get(): 
                
                # Captura evento de clicar em botão para fechar
                if event.type == pygame.QUIT:
                    return pygame.quit()
            
            # preencha a tela com a cor de fundo
            self.screen.fill(self.bgColor)
            
            # gera o desenho
            obj.draw(self);
            
            # aplica o antialiasing
            #self.meanFilter()
            
            # atualiza a tela 
            pygame.display.update()

            
        
        
    # retorna um vetor com o tamanho da tela
    def size(self):
        return (self.width, self.height)
    
    # modifica um pixel na tela com a cor desejada
    def setPixel(self, x, y, color):
        self.screen.set_at((x, y), color)
    
    # filtro da média para o antialising
    def meanFilter(self):
        # Captura a matrix da tela
        #frameBuffer2 = pygame.PixelArray(self.screen)
        
        from copy import copy
        frameBuffer = pygame.surfarray.array3d(self.screen)
        #print(frameBuffer)
        
        import numpy as np
        mask = np.ones((3, 3)) * 1/9 
        
        #print(mask)
        temp = np.zeros((3))
               
        for i in range(1, self.width - 1):
            for j in range(1, self.height - 1):    
                subI = frameBuffer[i-1:i+2,j-1:j+2, 0]
                temp[0] = np.sum(np.multiply(subI, mask))
                #display(temp)
                
                subI = frameBuffer[i-1:i+2,j-1:j+2, 1]
                temp[1] = np.sum(np.multiply(subI, mask))
                #display(temp)
                
                subI = frameBuffer[i-1:i+2,j-1:j+2, 2]
                temp[2] = np.sum(np.multiply(subI, mask))
                #display(temp)
                
                #temp = np.zeros((3))
                
                #for k in range(-1,2):
                #    for l in range(-1,2):
                #        for b in range(3):
                #            temp[b] = temp[b] + frameBuffer[i + k][j + l][b] * mask[k + 1][l + 1]
                        
                #print(pygame.Color(int(temp[0]), int(temp[1]), int(temp[2]), 255))
                        
                self.setPixel(i, j, pygame.Color(int(temp[0]), int(temp[1]), int(temp[2]), 255));
                #frameBuffer2[i][j] = pygame.Color(int(temp[0]), int(temp[1]), int(temp[2]), 255);

In [2]:
class Circle(object):
    # construtor da classe
    def __init__(self, xc, yc, raio, color, fill_color, estilo):
        self.xc = xc       # coordenada x do centro
        self.yc = yc       # coordenada y do centro
        self.raio = raio   # raio da circunferência
        self.color = color # cor do objeto
        self.fill_color = fill_color
        self.estilo = estilo
    
    # renderiza a circunferência desejada na tela
    # Algoritmo de Bresenham
    def draw(self, screen):
        # Definição e Inicialização de Variáveis locais      
        # Coniderando a circunferência ao redor da origem, mas renderizada transladada
        x = 0                # x inicial
        y = self.raio   # y inicial
        d = 1 - self.raio    # d de teste inicial

        tam_mat = len(self.estilo)

        lista_de_SEs = [] # Guarda locais em que caminho sudeste foi escolhido


        # Desenha os pontos inicias de cada quadrante
        self.drawCirclePoints(x, y, screen, self.color)

        # Gera os novos pontos e os renderiza
        while x < y:

            if d < 0:   # Direção E
                d = d + 2 * x + 3

                x = x + 1

                lista_de_SEs.append(0)

            else:       # Direção SE
                d = d + 2 * (x - y) + 5
                y = y - 1

                x = x + 1

                lista_de_SEs.append(1)

            self.drawCirclePoints(x, y, screen, self.color)


        # ---------------------------------------------
        copia_reversa = []
        for i in reversed(lista_de_SEs):
            copia_reversa.append(i)
       

        lista_tamanho_linhas = []
        try:
            index = lista_de_SEs.index(1)
            lista_tamanho_linhas.append(index)
        except:
            print('')
        lista_de_SEs[index] = 0

        for i in range(len(lista_de_SEs)):
            if(lista_de_SEs[i] ==  0):
                lista_tamanho_linhas.append(lista_tamanho_linhas[-1])
            else:
                try:
                    index = lista_de_SEs.index(1)
                    lista_tamanho_linhas.append(index)
                except:
                    continue

                lista_de_SEs[index] = 0
        

        # Crescimento no segundo Octante ---------------------------------

        
        for i in range(len(lista_de_SEs)):

            try:
                index = copia_reversa.index(1)
                lista_tamanho_linhas.append(index)
            except:
                continue
            
            
            copia_reversa[index] = 0
            
        
        # ESPELHAMENTO DA LISTA
        lista_tamanho_linhas.extend(reversed(lista_tamanho_linhas))
            
        #print(lista_tamanho_linhas)



        # ^^  teste -------------------------------------------------------------- ^^

        


        # PRIMEIRA PARTE
        teto = self.yc + (self.raio)
        chao = teto - 2*self.raio
        cont = 0
        for i in range(teto-1, chao+1,-1):

            tam_linha = lista_tamanho_linhas[cont]
            cont += 1

            for j in range(self.xc-tam_linha, self.xc+tam_linha):

                if self.estilo[j%len(self.estilo)-1][i%len(self.estilo)-1]:
                    #print(str(j) + " " + str(i))
                    screen.setPixel(j , i, self.fill_color)

        

            
    def drawCirclePoints(self, x, y, screen, color):
        xCentro = self.xc
        yCentro = self.yc
        screen.setPixel(xCentro + x, yCentro + y, color)
        screen.setPixel(xCentro + y, yCentro + x, color)
        screen.setPixel(xCentro + y, yCentro - x, color)
        screen.setPixel(xCentro + x, yCentro - y, color)
        screen.setPixel(xCentro - x, yCentro - y, color)
        screen.setPixel(xCentro - y, yCentro - x, color)
        screen.setPixel(xCentro - y, yCentro + x, color)
        screen.setPixel(xCentro - x, yCentro + y, color)

In [3]:
import pygame

pygame 2.1.2 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
pygame.init()  

(5, 0)

In [5]:
#screen = Screen("Tela", pygame.Color(255, 255, 255, 255),  900, 900)
#circ = Circle(450, 450, 400, pygame.Color(0, 0, 255, 255), pygame.Color(255,145,0,255), [[0,0,0,0,0],[1,1,1,1,0],[1,1,1,1,0],[1,1,1,1,0],[1,1,1,1,0]]);

#screen.run(circ);

In [6]:
class Picture(object):
    # Construtor da Classe
    def __init__(self):
        self.primitivas = [] # Define uma lista de primitivas para representar um desenho
    
    def draw(self, screen):
        
        circ = Circle(200, 200, 200, pygame.Color(0, 0, 255, 255), pygame.Color(255,145,0,255), [[0,0,0,0,0],[1,1,1,1,0],[1,1,1,1,0],[1,1,1,1,0],[1,1,1,1,0]]);
        circ2 = Circle(500, 500, 150, pygame.Color(0, 0, 255, 255), pygame.Color(255,145,0,255), [[1,1,1,1],[1,1,1,1],[0,0,0,0],[0,0,0,0]]);
        circ3 = Circle(800, 750, 100, pygame.Color(0, 0, 255, 255), pygame.Color(255,145,0,255), [[1,1,0,0],[1,1,0,0],[1,1,0,0],[1,1,0,0]]);

        
        # Insere primitivas na lista
        self.primitivas.append(circ);
        self.primitivas.append(circ2);  
        self.primitivas.append(circ3);      
        
        # Desenha cada primitiva que está na lista
        for item in self.primitivas:
            item.draw(screen);

In [7]:
screen = Screen("Tela", pygame.Color(255, 255, 255, 255), 1000, 1000)

# executa o desenho
pic = Picture();

screen.run(pic);